In [1]:
!pip install docx2python

   ---------------------------------------- 0.0/44.5 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.5 kB ? eta -:--:--
   ---------------------------------------- 44.5/44.5 kB 729.3 kB/s eta 0:00:00


In [18]:
!pip install python-docx

   ---------------------------------------- 0.0/244.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/244.3 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/244.3 kB 435.7 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/244.3 kB 939.4 kB/s eta 0:00:01
   -------------------------------------- - 235.5/244.3 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 244.3/244.3 kB 1.2 MB/s eta 0:00:00


In [12]:
from docx import Document
import pandas as pd
import glob
import os

In [13]:
data_folder = 'docs'

In [14]:
def get_data(folder):
    files = glob.glob(data_folder + '\*.docx')
    sections = []
    questions = []
    answers = []
    for file in files:
        section = os.path.basename(file)[:-5]
        doc = Document(file)
        current_answer = None
        for par in doc.paragraphs:
            if par.text.strip() == '':
                continue
            if par.style.name=='Heading 2':
                if current_answer:
                    answers.append(current_answer)
                if current_answer or current_answer is None:
                    questions.append(par.text)
                    sections.append(section)
                    current_answer = ''
            else:
                current_answer += par.text + '\n'
        answers.append(current_answer)
    return pd.DataFrame({'section': sections, 'question': questions, 'answer': answers})

In [15]:
ds = get_data(data_folder)
ds

,section,question,answer
0,Classic_models,What are various ways to predict a binary resp...,"Things to look at: N, P, linearly seperable?, ..."
1,Classic_models,What methods for solving linear regression do ...,"To solve linear regression, you need to find t..."
2,Classic_models,Nearest Neighbors Method,The nearest neighbors method (k-Nearest Neighb...
3,Classic_models,Nearest Neighbors Method in Real Applications,k-NN can serve as a good starting point (basel...
4,Classic_models,Pros and Cons of The nearest neighbors method,Pros:\nSimple implementation;\nWell studied;\n...
...,...,...,...
185,Statistics,If you had draws from a normal distribution wi...,A question like this tests your knowledge of t...
186,Statistics,How do you transform a Skewed Distribution int...,To transform a Skewed Distribution into a Norm...
187,Statistics,What's the difference between Binomial Distrib...,The Binomial distribution describes the probab...
188,Statistics,Bayesian Inference,Focusing solely on some statistic of the poste...


In [16]:
ds['question'][2]

'Nearest Neighbors Method'

In [17]:
ds['answer'][1]

"To solve linear regression, you need to find the coefficients\xa0which minimize the sum of squared errors.\nMatrix Algebra method: Let's say you have\xa0X, a matrix of features, and\xa0y, a vector with the values you want to predict. After going through the matrix algebra and minimization problem, you get this solution:\nBut solving this requires you to find an inverse, which can be time-consuming, if not impossible. Luckily, there are methods like Singular Value Decomposition (SVD) or QR Decomposition that can reliably calculate this part\xa0\xa0(called the pseudo-inverse) without actually needing to find an inverse. The popular python ML library\xa0sklearn\xa0uses SVD to solve least squares.\nAlternative method: Gradient Descent. \n"

In [18]:
ds_cleaned = ds.replace({'\u200d': '', '\xa0': ' ', '\u200b': ''}, regex=True)

In [19]:
ds_cleaned['question'][2]

'Nearest Neighbors Method'

In [20]:
ds_cleaned['answer'][1]

"To solve linear regression, you need to find the coefficients which minimize the sum of squared errors.\nMatrix Algebra method: Let's say you have X, a matrix of features, and y, a vector with the values you want to predict. After going through the matrix algebra and minimization problem, you get this solution:\nBut solving this requires you to find an inverse, which can be time-consuming, if not impossible. Luckily, there are methods like Singular Value Decomposition (SVD) or QR Decomposition that can reliably calculate this part  (called the pseudo-inverse) without actually needing to find an inverse. The popular python ML library sklearn uses SVD to solve least squares.\nAlternative method: Gradient Descent. \n"

In [21]:
ds_cleaned.to_csv('data/db.csv', index=False)